# Calculate Stats

In this jupyter notebook, we're calculating the total number of books scanned per center, per worker, and the rate of scanning per worker across all scanning centers found in the 9 million metadata records. The scanning centers are merged based on the geocoded dataset (i.e., boston and bpl are treated as the same scanning center because both are used for the scanning center located in the Boston Public Library in the metadata). 

In [1]:
import pandas as pd
import altair as alt


In [2]:
texts = pd.read_csv('/Volumes/Samsung_T5/scanning_labor_in_IA/geocoded-texts-data.csv', low_memory=False)

In [3]:
texts = texts.drop(columns=['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'])

## Scans per center per month

In [4]:
texts['month_year'] = pd.to_datetime(texts['scandate']).dt.to_period('M').astype('str')

In [5]:
scans_per_center = texts[['name', 'month_year', 'identifier']]

scans_per_center

,name,month_year,identifier
0,Datum Data Co. Ltd.,2013-10,geometrysuccessi00thom
1,Datum Data Co. Ltd.,2012-09,berdiemobilittde00hebe
2,Datum Data Co. Ltd.,2011-07,beilsteinshandb28beil
3,Datum Data Co. Ltd.,2012-08,theoryapplicatio00adva
4,Datum Data Co. Ltd.,2012-09,specialeditionus00edbo_0
...,...,...,...
9029407,Perpustakaan Provinsi Bali,2012-02,pratekaning-wong-mara-mandeg
9029408,Perpustakaan Provinsi Bali,2012-02,kramaning-caru-bang-bonggalan
9029409,Perpustakaan Provinsi Bali,2012-03,puja-parikrama-panigang-sasih
9029410,Perpustakaan Provinsi Bali,2012-03,puja-pancasanak-buda


In [6]:
scans_per_center = scans_per_center.groupby(['name', 'month_year']).count().reset_index()

In [15]:
scans_per_center_month = scans_per_center.loc[scans_per_center['month_year'] != 'NaT']


scans_per_center_month = scans_per_center_month.rename(columns={'name':'name', 'month_year': 'month_year', 'identifier':'books_scanned'})


In [16]:
scans_per_center_month.to_csv('/Users/e.schwartz/Documents/GitHub/ia_scanning_labor_data/csv_files/scans_per_center_per_month.csv')

In [17]:
# missing data where scan dates were lost
missing_scans_per_month = scans_per_center.loc[scans_per_center['month_year'] == 'NaT'].reset_index()
missing_scans_per_month.to_csv('/Users/e.schwartz/Documents/GitHub/ia_scanning_labor_data/scans_missing_timestamp_per_center.csv')

In [22]:
# scans per month as a line chart

import altair as alt

alt.Chart("https://raw.githubusercontent.com/ers6/ia_scanning_labor_data/main/csv_files/scans_per_center_per_month.csv").mark_line().encode(
    x='month_year:N',
    y='books_scanned:Q',
    color='name:N'
).interactive()

alt.Chart(...)

In [103]:
# scans per month as a stacked bar chart

selector = alt.selection_single(encodings=['color'])

alt.Chart(scans_per_center_month).mark_bar().encode(
    x='month_year',
    y='sum(identifier)',
    color=alt.condition(selector, 'name', alt.value('lightgray'))
).add_params(selector)

/Users/e.schwartz/anaconda3/envs/data_analyzer/lib/python3.9/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)

## Calculating percentages of the total 

In [23]:
total_scans = scans_per_center.groupby(['name']).sum().reset_index()

/var/folders/4s/xckhb7kj6bqchm0hf_c4c1m40000gr/T/ipykernel_37440/1250914918.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_scans = scans_per_center.groupby(['name']).sum().reset_index()


In [24]:
total_scans['identifier'].sum()

9029412

In [25]:
total_scans['percent_of_total'] = 0

In [26]:
i = 0
for i in range(len(total_scans)):
    total_scans.at[i,'percent_of_total'] = int(total_scans.at[i, 'identifier']) / 9029412
    
    

In [27]:
total_scans = total_scans.rename(columns={'name': 'scan_center', 'identifier': 'total_scans', 'percent_of_total':'percent_of_total'})
total_scans['percent_of_total'] = total_scans['percent_of_total'].map('{:.10%}'.format)

In [28]:

source = total_scans

base = alt.Chart(source).encode(
    x= alt.X('total_scans:Q').scale(type='log'),
    y="scan_center:N",
    color = 'scan_center:N'

)
base.mark_bar() 

alt.Chart(...)

In [29]:

total_scans.to_csv('/Volumes/Samsung_T5/scanning_labor_in_IA/total_scans.csv')

In [30]:
total_scans.sort_values(by=['percent_of_total'], ascending=False)

,scan_center,total_scans,percent_of_total
30,"Innodata Knowledge Services, Inc.",5440127,60.2489619479%
20,Datum Data Co. Ltd.,482936,5.3484767336%
72,University of Toronto,465884,5.1596272271%
28,Hong Kong,293756,3.2533236937%
10,Boston Public Library,256899,2.8451354307%
...,...,...,...
51,Research Institute of Korean Studies,7,0.0000775244%
45,Peabody Essex Museum,7,0.0000775244%
61,Trent University,2,0.0000221498%
37,Missouri Botanical Garden,1,0.0000110749%


### Analysis by total numbers alone
By total scans alone, from the 9 million records we captured, 60% of them were scanned at Innodata Knowledge Services, a business process outsourcing firm in Mandeau City, Philippines. This center has only been opened since 2018. The second most common scanning center, Datum Data, Co., accounts for only 5.34% of the total records. It, too, is a BPO firm in the global South. The university of Toronto library accounts for 5.15% of the scans--being the only academic library to break the top 5. This makes sense because Toronto was the first university partner. Hong Kong accounts for 3.25% of the total -- another outsourced center in China. And then regional scanning centers and those founded with grants come in.

That being said, the total numbers of books scanned, alone, do not tell a full story. After all, the UoT center has been in operation since 2004. The Innodata center was opened in 2018. We need to understand the rate of scans to get a better idea of the amount of work being done per center.

## Calculating Average Rate of Work per Center per Month

In [7]:

scans_per_center_month = pd.read_csv("https://raw.githubusercontent.com/ers6/ia_scanning_labor_data/main/csv_files/scans_per_center_per_month.csv")
average_scans_per_month = scans_per_center_month.groupby(['name']).mean()


average_scans_per_month


/var/folders/4s/xckhb7kj6bqchm0hf_c4c1m40000gr/T/ipykernel_84102/1771829147.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  average_scans_per_month = scans_per_center_month.groupby(['name']).mean()


,Unnamed: 0,books_scanned
name,,
Allen County Public Library Geneaology Center,91.0,1165.248619
American Museum of Natural History,188.5,2.416667
American Numismatic Society,237.5,180.848837
American Printing House for the Blind,305.0,49.468085
Analysis and Policy Observatory (APO),332.0,8.200000
...,...,...
University of Victoria,4746.5,90.026316
University of Warwick,4799.5,25.500000
Washington University in St. Louis,4858.0,208.045977


In [6]:
average_scans_per_month.sort_values(by=['books_scanned'], ascending=False)



,Unnamed: 0,books_scanned
name,,
"Innodata Knowledge Services, Inc.",1938.0,57419.929825
Hong Kong,1871.5,4447.909091
Datum Data Co. Ltd.,1349.5,3655.007576
University of Toronto,4608.0,2295.437186
British Library,846.0,2153.028037
...,...,...
American Museum of Natural History,188.5,2.416667
Peabody Essex Museum,2775.0,2.000000
Research Institute of Korean Studies,2994.5,1.750000


In [8]:
average_scans_per_month.rename(columns={'name':'name', 'books_scanned': 'avg_monthly_scans'})
average_scans_per_month.to_csv('/Volumes/Samsung_T5/scanning_labor_in_IA/average_scans_per_month.csv')

### Analyzing the average number of scans per month
Taking the average number of books scanned into account, the 3 outsourced centers take the top spot. 

## Rate of work

This still doesn't necessarily tell us the whole story. Perhaps the outsourced centers simply had way more workers than the others. We need to understand how many books are being scanned per worker per day across the centers. When possible, we can also consider the number of images captured across (image count) per worker per day to take the length of the texts scanned into account. 

In [9]:

texts['scan_day'] = pd.to_datetime(texts['scandate']).dt.to_period('D').astype('str')
texts['operator'] = texts['operator'].astype('str')
texts['operator'] = texts['operator'].map(str.lower)
scans_per_day = texts[['name', 'scan_day', 'identifier', 'operator']]


In [10]:
scans_per_worker_day = scans_per_day.groupby(['name','operator', 'scan_day']).count().reset_index()

In [11]:
scans_per_worker_day

,name,operator,scan_day,identifier
0,1 Dollar Scan,nan,NaT,20
1,Allen County Public Library Geneaology Center,admin-jeff-sharpe@archive.org,2008-06-03,1
2,Allen County Public Library Geneaology Center,admin-jeff-sharpe@archive.org,2008-06-09,1
3,Allen County Public Library Geneaology Center,admin-jeff-sharpe@archive.org,2008-07-15,1
4,Allen County Public Library Geneaology Center,admin-jeff-sharpe@archive.org,2008-07-16,1
...,...,...,...,...
429349,Yiddish Book Center,volunteer-zoe-schacht-levine@archive.org,2013-04-22,9
429350,Yiddish Book Center,volunteer-zoe-schacht-levine@archive.org,2013-04-24,6
429351,Yiddish Book Center,volunteer-zoe-schacht-levine@archive.org,2013-04-25,3
429352,Yiddish Book Center,volunteer-zoe-schacht-levine@archive.org,2013-04-29,4


In [22]:
# weeding out missing data- we want to throw out rows where operator = nan and scanday = nat


scans_per_worker_day = scans_per_worker_day.loc[scans_per_worker_day['scan_day'] != 'NaT']

scans_per_worker_day = scans_per_worker_day.loc[scans_per_worker_day['operator'] != 'nan']


In [24]:

scans_per_worker_day = scans_per_worker_day.rename(columns={'name':'name', 'operator': 'operator','scan_day':'scan_day','identifier':'books_scanned'})
scans_per_worker_day.to_csv('/Users/e.schwartz/Documents/GitHub/ia_scanning_labor_data/csv_files/books_scanned_per_worker_per_day.csv')


In [25]:
operators = scans_per_worker_day['operator'].unique().tolist()

In [27]:
avg_scans_per_worker = scans_per_worker_day.groupby(['name', 'operator']).mean().sort_values(by='books_scanned', ascending=False)

avg_scans_per_worker

/var/folders/4s/xckhb7kj6bqchm0hf_c4c1m40000gr/T/ipykernel_84102/2518951010.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg_scans_per_worker = scans_per_worker_day.groupby(['name', 'operator']).mean().sort_values(by='books_scanned', ascending=False)


books_scanned
name                             operator                                           
Boston Public Library            book-creator-script@archive.org          329.375000
                                 partner                                  200.000000
BYU, Provo                       partner                                  184.000000
Church History Library           partner                                  174.000000
State Library of Pennsylvania    tt scribe                                 95.333333
...                                                                              ...
Internt Archive Physical Archive scanner-august-bourre@archive.org          1.000000
                                 scanner-andrew-pasternak@archive.org       1.000000
University of Toronto            august@archive.org                         1.000000
Hong Kong                        associate-lan-zhu@archive.org              1.000000
Zhejiang University              ken@archive.org                            1.000000

[3179 rows x 1 columns]

In [28]:
avg_scans_per_worker.to_csv('/Volumes/Samsung_T5/scanning_labor_in_IA/avg_scans_per_worker.csv')

In [18]:
avg_scans_per_worker

books_scanned
name                             operator                                           
Boston Public Library            book-creator-script@archive.org          329.375000
                                 partner                                  200.000000
BYU, Provo                       partner                                  184.000000
Church History Library           partner                                  174.000000
State Library of Pennsylvania    tt scribe                                 95.333333
...                                                                              ...
Internt Archive Physical Archive scanner-august-bourre@archive.org          1.000000
                                 scanner-andrew-pasternak@archive.org       1.000000
University of Toronto            august@archive.org                         1.000000
Hong Kong                        associate-lan-zhu@archive.org              1.000000
Zhejiang University              ken@archive.org                            1.000000

[3179 rows x 1 columns]

### Analyzing avg scans per worker per day
there are some accounts in here that probably aren't tied to one person/are due to batch processing or some really short documents. However, across the board we start to see that workers at Datum Data, Hong Kong, and Innodata scan more books on average each day than workers at other centers. 

need to get the number of people working at each center each day and the number of books scanned each day to get a rate of scanning proportionate to workers employed

## Avg. daily scans per employees at centers over all


In [46]:
books =  texts[['name', 'scan_day', 'identifier', 'operator']]

In [47]:
missing_books = books.loc[books['scan_day'] == 'NaT']

missing_books = books.loc[books['operator'] == 'nan']

missing_books

,name,scan_day,identifier,operator
764,Datum Data Co. Ltd.,2011-11-23,snowflakecutuses00sibb,nan
1697,Datum Data Co. Ltd.,NaT,chaffonwindnovel00dibb,nan
2083,Datum Data Co. Ltd.,NaT,whatdoyouwantfro00dorr,nan
2167,Datum Data Co. Ltd.,NaT,bigballadjambore00davi,nan
2215,Datum Data Co. Ltd.,NaT,canadianshortsto00mich,nan
...,...,...,...,...
9029407,Perpustakaan Provinsi Bali,2012-02-23,pratekaning-wong-mara-mandeg,nan
9029408,Perpustakaan Provinsi Bali,2012-02-23,kramaning-caru-bang-bonggalan,nan
9029409,Perpustakaan Provinsi Bali,2012-03-12,puja-parikrama-panigang-sasih,nan
9029410,Perpustakaan Provinsi Bali,2012-03-12,puja-pancasanak-buda,nan


In [48]:
books = books.loc[books['scan_day'] != 'NaT']
books = books.loc[books['operator'] != 'nan']
books

,name,scan_day,identifier,operator
0,Datum Data Co. Ltd.,2013-10-10,geometrysuccessi00thom,scanner-shenzhen-leo@archive.org
1,Datum Data Co. Ltd.,2012-09-08,berdiemobilittde00hebe,scanner-shenzhen-mary@archive.org
2,Datum Data Co. Ltd.,2011-07-16,beilsteinshandb28beil,scanner-shenzhen-lina@archive.org
3,Datum Data Co. Ltd.,2012-08-14,theoryapplicatio00adva,scanner-shenzhen-dragon@archive.org
4,Datum Data Co. Ltd.,2012-09-21,specialeditionus00edbo_0,scanner-shenzhen-yan@archive.org
...,...,...,...,...
9028846,New York Botanical Garden,2017-12-08,lrobokibotanikaf01agar,operator1.nybg@archive.org
9028847,New York Botanical Garden,2017-04-05,alphonsedecandol00cand,operator1.nybg@archive.org
9028848,New York Botanical Garden,2017-12-20,memoirsoftorreyb2019torr,operator1.nybg@archive.org
9028849,New York Botanical Garden,2017-05-11,bulletinofpopula1191arno,operator1.nybg@archive.org


In [217]:
# number of people working for each day that there was at least 1 book scanned

books_per_worker = books[['name', 'operator', 'scan_day']]
# .groupby(['name', 'operator', 'scan_day']).count().sort_values(by='identifier', ascending=False)
books_per_worker = books_per_worker.drop_duplicates()
books_per_worker = books_per_worker.groupby(['name', 'scan_day']).count().reset_index()
books_per_worker

,name,scan_day,operator
0,Allen County Public Library Geneaology Center,2002-01-09,2
1,Allen County Public Library Geneaology Center,2002-01-10,1
2,Allen County Public Library Geneaology Center,2002-01-15,1
3,Allen County Public Library Geneaology Center,2008-05-15,1
4,Allen County Public Library Geneaology Center,2008-05-16,2
...,...,...,...
73314,Yiddish Book Center,2013-07-19,1
73315,Yiddish Book Center,2013-07-26,1
73316,Yiddish Book Center,2013-07-29,1
73317,Yiddish Book Center,2013-08-02,1


In [219]:
# number of books scanned each day
books_per_day = books[['name', 'scan_day', 'identifier']]

books_per_day = books_per_day.groupby(['name', 'scan_day']).count().reset_index()

books_per_day

,name,scan_day,identifier
0,Allen County Public Library Geneaology Center,2002-01-09,2
1,Allen County Public Library Geneaology Center,2002-01-10,1
2,Allen County Public Library Geneaology Center,2002-01-15,1
3,Allen County Public Library Geneaology Center,2008-05-15,2
4,Allen County Public Library Geneaology Center,2008-05-16,2
...,...,...,...
73314,Yiddish Book Center,2013-07-19,1
73315,Yiddish Book Center,2013-07-26,9
73316,Yiddish Book Center,2013-07-29,2
73317,Yiddish Book Center,2013-08-02,8


In [221]:
# merging the dataframes 

books_stats = pd.merge(books_per_worker, books_per_day, how='inner', on=['name', 'scan_day'])

books_stats



,name,scan_day,operator,identifier
0,Allen County Public Library Geneaology Center,2002-01-09,2,2
1,Allen County Public Library Geneaology Center,2002-01-10,1,1
2,Allen County Public Library Geneaology Center,2002-01-15,1,1
3,Allen County Public Library Geneaology Center,2008-05-15,1,2
4,Allen County Public Library Geneaology Center,2008-05-16,2,2
...,...,...,...,...
73314,Yiddish Book Center,2013-07-19,1,1
73315,Yiddish Book Center,2013-07-26,1,9
73316,Yiddish Book Center,2013-07-29,1,2
73317,Yiddish Book Center,2013-08-02,1,8


In [224]:
books_stats['books_per_worker'] = ''
i = 0
for i in range(len(books_stats)): 
    books_stats.at[i,'books_per_worker'] = float(books_stats.at[i, 'identifier']) / float(books_stats.at[i, 'operator']) 
    i += 0

In [251]:
books_stats.sort_values(by='books_per_worker', ascending=False)

books_stats

books_stats.to_csv('/Volumes/Samsung_T5/scanning_labor_in_IA/books_scanned_per_worker_per_day.csv')


This is the average of the average number of books scanned per employee at each center. How helpful is this-- i'm not sure. the length of the books may be a better metric to understand the rate of work. The national agricultural library, for example, may not have a ton of lengthy books. 

## Pages scanned per worker per day

In [155]:

pages = texts[['name', 'scan_day', 'imagecount', 'operator']]

In [156]:
pages

,name,scan_day,imagecount,operator
0,Datum Data Co. Ltd.,2013-10-10,200.0,scanner-shenzhen-leo@archive.org
1,Datum Data Co. Ltd.,2012-09-08,250.0,scanner-shenzhen-mary@archive.org
2,Datum Data Co. Ltd.,2011-07-16,962.0,scanner-shenzhen-lina@archive.org
3,Datum Data Co. Ltd.,2012-08-14,590.0,scanner-shenzhen-dragon@archive.org
4,Datum Data Co. Ltd.,2012-09-21,888.0,scanner-shenzhen-yan@archive.org
...,...,...,...,...
9029407,Perpustakaan Provinsi Bali,2012-02-23,NaN,nan
9029408,Perpustakaan Provinsi Bali,2012-02-23,NaN,nan
9029409,Perpustakaan Provinsi Bali,2012-03-12,NaN,nan
9029410,Perpustakaan Provinsi Bali,2012-03-12,NaN,nan


In [165]:
missing_pages = pages.loc[pages['scan_day'] == 'NaT']

missing_pages = pages.loc[pages['operator'] == 'nan']
# missing_pages = pages.loc[pages['imagecount'] == 'NaN']

missing_pages

,name,scan_day,imagecount,operator
764,Datum Data Co. Ltd.,2011-11-23,70.0,nan
1697,Datum Data Co. Ltd.,NaT,214.0,nan
2083,Datum Data Co. Ltd.,NaT,186.0,nan
2167,Datum Data Co. Ltd.,NaT,314.0,nan
2215,Datum Data Co. Ltd.,NaT,142.0,nan
...,...,...,...,...
9029407,Perpustakaan Provinsi Bali,2012-02-23,NaN,nan
9029408,Perpustakaan Provinsi Bali,2012-02-23,NaN,nan
9029409,Perpustakaan Provinsi Bali,2012-03-12,NaN,nan
9029410,Perpustakaan Provinsi Bali,2012-03-12,NaN,nan


### what are we missing? 

2234140 entries don't have a scanning date field, so we don't know anything about them. these are from 67/70 scanning centers- 

2270554 have a missing operator field 

--- we should see if these errors are more common across different scanning centers at some point 

but for now let's proceed with the data we do have

In [166]:
# weeding out missing data- we want to throw out rows where operator = nan and scanday = nat

pages = pages.loc[pages['scan_day'] != 'NaT']

pages = pages.loc[pages['operator'] != 'nan']
pages = pages.loc[pages['imagecount'] != 'NaN']




In [241]:
# note that there's a lot of missing data when we drop these rows 
workers = pages[['name', 'scan_day', 'operator']]
workers = workers.drop_duplicates()
workers = workers.groupby(['name', 'scan_day']).count().reset_index()



In [242]:
# number of workers working each day we have data for 
workers

,name,scan_day,operator
0,Allen County Public Library Geneaology Center,2002-01-09,2
1,Allen County Public Library Geneaology Center,2002-01-10,1
2,Allen County Public Library Geneaology Center,2002-01-15,1
3,Allen County Public Library Geneaology Center,2008-05-15,1
4,Allen County Public Library Geneaology Center,2008-05-16,2
...,...,...,...
73314,Yiddish Book Center,2013-07-19,1
73315,Yiddish Book Center,2013-07-26,1
73316,Yiddish Book Center,2013-07-29,1
73317,Yiddish Book Center,2013-08-02,1


In [244]:
pages_per_day = pages[['name', 'scan_day', 'imagecount']]
pages_per_day = pages_per_day.groupby(['name', 'scan_day']).sum()
pages_per_day = pages_per_day.reset_index()

pages_per_day

,name,scan_day,imagecount
0,Allen County Public Library Geneaology Center,2002-01-09,752.0
1,Allen County Public Library Geneaology Center,2002-01-10,108.0
2,Allen County Public Library Geneaology Center,2002-01-15,506.0
3,Allen County Public Library Geneaology Center,2008-05-15,564.0
4,Allen County Public Library Geneaology Center,2008-05-16,454.0
...,...,...,...
73314,Yiddish Book Center,2013-07-19,114.0
73315,Yiddish Book Center,2013-07-26,1663.0
73316,Yiddish Book Center,2013-07-29,180.0
73317,Yiddish Book Center,2013-08-02,1009.0


In [183]:
# operators_per_day = pages.groupby(['name', 'scan_day']).count()
# operators_per_day = operators_per_day.drop(columns=['imagecount']).reset_index()

# operators_per_day

,name,scan_day,operator
0,Allen County Public Library Geneaology Center,2002-01-09,2
1,Allen County Public Library Geneaology Center,2002-01-10,1
2,Allen County Public Library Geneaology Center,2002-01-15,1
3,Allen County Public Library Geneaology Center,2008-05-15,2
4,Allen County Public Library Geneaology Center,2008-05-16,2
...,...,...,...
73314,Yiddish Book Center,2013-07-19,1
73315,Yiddish Book Center,2013-07-26,9
73316,Yiddish Book Center,2013-07-29,2
73317,Yiddish Book Center,2013-08-02,8


In [245]:
# need to join where name and scanday overlap 

pages_stats = pd.merge(pages_per_day, workers, how='inner', on=['name', 'scan_day'])



In [246]:
pages_stats

,name,scan_day,imagecount,operator
0,Allen County Public Library Geneaology Center,2002-01-09,752.0,2
1,Allen County Public Library Geneaology Center,2002-01-10,108.0,1
2,Allen County Public Library Geneaology Center,2002-01-15,506.0,1
3,Allen County Public Library Geneaology Center,2008-05-15,564.0,1
4,Allen County Public Library Geneaology Center,2008-05-16,454.0,2
...,...,...,...,...
73314,Yiddish Book Center,2013-07-19,114.0,1
73315,Yiddish Book Center,2013-07-26,1663.0,1
73316,Yiddish Book Center,2013-07-29,180.0,1
73317,Yiddish Book Center,2013-08-02,1009.0,1


In [247]:
pages_stats.sort_values(by='operator', ascending=False)

,name,scan_day,imagecount,operator
30697,"Innodata Knowledge Services, Inc.",2022-05-21,1104638.0,151
30668,"Innodata Knowledge Services, Inc.",2022-04-22,1213972.0,151
30667,"Innodata Knowledge Services, Inc.",2022-04-21,1046634.0,150
30930,"Innodata Knowledge Services, Inc.",2023-01-11,1272916.0,150
30931,"Innodata Knowledge Services, Inc.",2023-01-12,1209358.0,148
...,...,...,...,...
48457,State Library of Pennsylvania,2016-09-28,2876.0,1
48456,State Library of Pennsylvania,2016-09-27,1254.0,1
48455,State Library of Pennsylvania,2016-09-22,1174.0,1
48454,State Library of Pennsylvania,2016-09-21,1188.0,1


In [248]:
pages_stats['pages_per_worker'] = ''

In [249]:
i = 0
for i in range(len(result)): 
    pages_stats.at[i,'pages_per_worker'] = float(pages_stats.at[i, 'imagecount']) / float(pages_stats.at[i, 'operator']) 
    i += 0

In [250]:
pages_stats.sort_values(by='pages_per_worker', ascending=False)

,name,scan_day,imagecount,operator,pages_per_worker
21194,Datum Data Co. Ltd.,2013-10-17,101942.0,1,101942.0
63087,University of Alberta,2010-01-25,86600.0,1,86600.0
21197,Datum Data Co. Ltd.,2013-10-22,84322.0,1,84322.0
63756,University of Alberta,2013-02-05,82095.0,1,82095.0
21199,Datum Data Co. Ltd.,2013-10-24,78542.0,1,78542.0
...,...,...,...,...,...
28131,Getty Research Institute Valencia Warehouse,2023-05-25,0.0,1,0.0
28129,Getty Research Institute Valencia Warehouse,2023-05-23,0.0,1,0.0
31512,Internet Archive Sheridan Headquarters,2008-04-18,0.0,1,0.0
28127,Getty Research Institute Valencia Warehouse,2023-05-20,0.0,1,0.0


In [252]:
pages_per_worker.to_csv('/Volumes/Samsung_T5/scanning_labor_in_IA/pages_scanned_per_worker_per_day.csv')

In [254]:
workers.to_csv('/Volumes/Samsung_T5/scanning_labor_in_IA/number_workers_per_day.csv')

## Days worked per worker

In [332]:
operators = texts[['name', 'operator']]

In [333]:
# remove missing data
operators = operators.loc[operators['operator'] != 'nan']



In [334]:
operators = operators.drop_duplicates()

In [335]:
operators ['days_worked'] = ''
operators['first_day'] = ''
operators['last_day'] = ''

In [336]:
operators = operators.reset_index()

In [341]:
workers_days = texts[['name', 'operator', 'scan_day']]
workers_days['scan_day'] = pd.to_datetime(workers_days['scan_day']).dt.to_period('D')

workers_days = workers_days.loc[workers_days['operator'] != 'nan']
workers_days = workers_days.loc[workers_days['scan_day'] != 'NaT']

workers_days = workers_days.drop_duplicates()


/var/folders/4s/xckhb7kj6bqchm0hf_c4c1m40000gr/T/ipykernel_1574/2801117023.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  workers_days['scan_day'] = pd.to_datetime(workers_days['scan_day']).dt.to_period('D')


In [342]:
workers_days

,name,operator,scan_day
0,Datum Data Co. Ltd.,scanner-shenzhen-leo@archive.org,2013-10-10
1,Datum Data Co. Ltd.,scanner-shenzhen-mary@archive.org,2012-09-08
2,Datum Data Co. Ltd.,scanner-shenzhen-lina@archive.org,2011-07-16
3,Datum Data Co. Ltd.,scanner-shenzhen-dragon@archive.org,2012-08-14
4,Datum Data Co. Ltd.,scanner-shenzhen-yan@archive.org,2012-09-21
...,...,...,...
9028837,New York Botanical Garden,operator1.nybg@archive.org,2017-12-20
9028844,New York Botanical Garden,operator1.nybg@archive.org,2017-12-07
9028846,New York Botanical Garden,operator1.nybg@archive.org,2017-12-08
9028849,New York Botanical Garden,operator1.nybg@archive.org,2017-05-11


In [343]:
def get_days_worked(scan_center, worker): 
    min_date = workers_days.loc[ (workers_days['name'] == scan_center) & (workers_days['operator'] == worker)]['scan_day'].min()
    print('mindate:', min_date)
    max_date = workers_days.loc[ (workers_days['name'] == scan_center) & (workers_days['operator'] == worker)]['scan_day'].max()
    print('maxdate:', max_date)
    return{'days_worked':(max_date - min_date)/ pd.Timedelta(days=1),
          'min_date': min_date,
          'max_date': max_date}
#     return min_date

In [344]:
i = 0 
for i in range(len(operators)):
#     print(operators.iloc[i]['operator'])
#     print(operators.iloc[i]['name'])
    days_worked = get_days_worked(operators.iloc[i]['name'], operators.iloc[i]['operator'])
    print(days_worked)
    operators.loc[i, 'days_worked'] = days_worked['days_worked']
    operators.loc[i, 'first_day'] = days_worked['min_date']
    operators.loc[i, 'last_day'] = days_worked['max_date']
    print(operators.iloc[i])
    print(i)
    print('--------------------------------')
    
    
    

mindate: 2010-08-26
maxdate: 2016-04-01
{'days_worked': 2045.0, 'min_date': Period('2010-08-26', 'D'), 'max_date': Period('2016-04-01', 'D')}
index                                         0
name                        Datum Data Co. Ltd.
operator       scanner-shenzhen-leo@archive.org
days_worked                              2045.0
first_day                            2010-08-26
last_day                             2016-04-01
Name: 0, dtype: object
0
--------------------------------
mindate: 2010-12-23
maxdate: 2013-05-27
{'days_worked': 886.0, 'min_date': Period('2010-12-23', 'D'), 'max_date': Period('2013-05-27', 'D')}
index                                          1
name                         Datum Data Co. Ltd.
operator       scanner-shenzhen-mary@archive.org
days_worked                                886.0
first_day                             2010-12-23
last_day                              2013-05-27
Name: 1, dtype: object
1
--------------------------------
mindate: 2010-12-24

maxdate: 2012-10-20
{'days_worked': 148.0, 'min_date': Period('2012-05-25', 'D'), 'max_date': Period('2012-10-20', 'D')}
index                                         25
name                         Datum Data Co. Ltd.
operator       scanner-shenzhen-song@archive.org
days_worked                                148.0
first_day                             2012-05-25
last_day                              2012-10-20
Name: 17, dtype: object
17
--------------------------------
mindate: 2011-07-26
maxdate: 2013-01-19
{'days_worked': 543.0, 'min_date': Period('2011-07-26', 'D'), 'max_date': Period('2013-01-19', 'D')}
index                                        26
name                        Datum Data Co. Ltd.
operator       scanner-shenzhen-roy@archive.org
days_worked                               543.0
first_day                            2011-07-26
last_day                             2013-01-19
Name: 18, dtype: object
18
--------------------------------
mindate: 2012-02-10
maxdate: 2012-10

maxdate: 2012-06-27
{'days_worked': 132.0, 'min_date': Period('2012-02-16', 'D'), 'max_date': Period('2012-06-27', 'D')}
index                                          60
name                          Datum Data Co. Ltd.
operator       scanner-shenzhen-wendy@archive.org
days_worked                                 132.0
first_day                              2012-02-16
last_day                               2012-06-27
Name: 34, dtype: object
34
--------------------------------
mindate: 2011-08-01
maxdate: 2012-10-23
{'days_worked': 449.0, 'min_date': Period('2011-08-01', 'D'), 'max_date': Period('2012-10-23', 'D')}
index                                         61
name                         Datum Data Co. Ltd.
operator       scanner-shenzhen-king@archive.org
days_worked                                449.0
first_day                             2011-08-01
last_day                              2012-10-23
Name: 35, dtype: object
35
--------------------------------
mindate: 2012-03-22
maxd

mindate: 2011-07-27
maxdate: 2011-12-23
{'days_worked': 149.0, 'min_date': Period('2011-07-27', 'D'), 'max_date': Period('2011-12-23', 'D')}
index                                        141
name                         Datum Data Co. Ltd.
operator       scanner-shenzhen-mark@archive.org
days_worked                                149.0
first_day                             2011-07-27
last_day                              2011-12-23
Name: 52, dtype: object
52
--------------------------------
mindate: 2012-05-15
maxdate: 2012-09-01
{'days_worked': 109.0, 'min_date': Period('2012-05-15', 'D'), 'max_date': Period('2012-09-01', 'D')}
index                                         156
name                          Datum Data Co. Ltd.
operator       scanner-shenzhen-betty@archive.org
days_worked                                 109.0
first_day                              2012-05-15
last_day                               2012-09-01
Name: 53, dtype: object
53
--------------------------------
mind

maxdate: 2012-04-29
{'days_worked': 61.0, 'min_date': Period('2012-02-28', 'D'), 'max_date': Period('2012-04-29', 'D')}
index                                       595
name                        Datum Data Co. Ltd.
operator       scanner-shenzhen-bin@archive.org
days_worked                                61.0
first_day                            2012-02-28
last_day                             2012-04-29
Name: 69, dtype: object
69
--------------------------------
mindate: 2010-12-27
maxdate: 2013-01-03
{'days_worked': 738.0, 'min_date': Period('2010-12-27', 'D'), 'max_date': Period('2013-01-03', 'D')}
index                                          755
name                           Datum Data Co. Ltd.
operator       scanner-shenzhen-thomas@archive.org
days_worked                                  738.0
first_day                               2010-12-27
last_day                                2013-01-03
Name: 70, dtype: object
70
--------------------------------
mindate: 2011-08-19
maxda

...

maxdate: 2020-01-07
{'days_worked': 949.0, 'min_date': Period('2017-06-02', 'D'), 'max_date': Period('2020-01-07', 'D')}
index                                 8960946
name                   University of Victoria
operator       ttscribe1.victoria@archive.org
days_worked                             949.0
first_day                          2017-06-02
last_day                           2020-01-07
Name: 3136, dtype: object
3136
--------------------------------
mindate: 2020-01-08
maxdate: 2023-06-02
{'days_worked': 1241.0, 'min_date': Period('2020-01-08', 'D'), 'max_date': Period('2023-06-02', 'D')}
index                         8960954
name           University of Victoria
operator         digitization@uvic.ca
days_worked                    1241.0
first_day                  2020-01-08
last_day                   2023-06-02
Name: 3137, dtype: object
3137
--------------------------------
mindate: 2020-10-08
maxdate: 2020-10-09
{'days_worked': 1.0, 'min_date': Period('2020-10-08', 'D'), 'max_

mindate: 2011-11-03
maxdate: 2012-10-22
{'days_worked': 354.0, 'min_date': Period('2011-11-03', 'D'), 'max_date': Period('2012-10-22', 'D')}
index                          8999222
name            Church History Library
operator       spitzerba@ldschurch.org
days_worked                      354.0
first_day                   2011-11-03
last_day                    2012-10-22
Name: 3154, dtype: object
3154
--------------------------------
mindate: 2009-09-08
maxdate: 2010-03-09
{'days_worked': 182.0, 'min_date': Period('2009-09-08', 'D'), 'max_date': Period('2010-03-09', 'D')}
index                           8999268
name             Church History Library
operator       smithap@familysearch.org
days_worked                       182.0
first_day                    2009-09-08
last_day                     2010-03-09
Name: 3155, dtype: object
3155
--------------------------------
mindate: 2009-07-29
maxdate: 2009-10-19
{'days_worked': 82.0, 'min_date': Period('2009-07-29', 'D'), 'max_date': Per

maxdate: 2010-10-12
{'days_worked': 18.0, 'min_date': Period('2010-09-24', 'D'), 'max_date': Period('2010-10-12', 'D')}
index                         9002020
name           Church History Library
operator        yvettem@ldschurch.org
days_worked                      18.0
first_day                  2010-09-24
last_day                   2010-10-12
Name: 3174, dtype: object
3174
--------------------------------
mindate: 2009-11-19
maxdate: 2010-02-11
{'days_worked': 84.0, 'min_date': Period('2009-11-19', 'D'), 'max_date': Period('2010-02-11', 'D')}
index                          9004079
name            Church History Library
operator       hallsr@familysearch.org
days_worked                       84.0
first_day                   2009-11-19
last_day                    2010-02-11
Name: 3175, dtype: object
3175
--------------------------------
mindate: 2012-07-20
maxdate: 2012-07-20
{'days_worked': 0.0, 'min_date': Period('2012-07-20', 'D'), 'max_date': Period('2012-07-20', 'D')}
index      

maxdate: 2013-03-12
{'days_worked': 197.0, 'min_date': Period('2012-08-27', 'D'), 'max_date': Period('2013-03-12', 'D')}
index                                       9021425
name                                     BYU, Provo
operator       volunteer-lindsay-weaver@archive.org
days_worked                                   197.0
first_day                                2012-08-27
last_day                                 2013-03-12
Name: 3191, dtype: object
3191
--------------------------------
mindate: 2013-06-17
maxdate: 2013-06-18
{'days_worked': 1.0, 'min_date': Period('2013-06-17', 'D'), 'max_date': Period('2013-06-18', 'D')}
index                                    9021739
name                                  BYU, Provo
operator       admin-gemma-waterston@archive.org
days_worked                                  1.0
first_day                             2013-06-17
last_day                              2013-06-18
Name: 3192, dtype: object
3192
--------------------------------
mindat

mindate: 2023-04-26
maxdate: 2023-06-09
{'days_worked': 44.0, 'min_date': Period('2023-04-26', 'D'), 'max_date': Period('2023-06-09', 'D')}
index                                               9028612
name                                Hamilton Public Library
operator       associate-emily-skewes-donaldson@archive.org
days_worked                                            44.0
first_day                                        2023-04-26
last_day                                         2023-06-09
Name: 3209, dtype: object
3209
--------------------------------
mindate: 2016-11-29
maxdate: 2020-03-06
{'days_worked': 1193.0, 'min_date': Period('2016-11-29', 'D'), 'max_date': Period('2020-03-06', 'D')}
index                             9028813
name            New York Botanical Garden
operator       operator1.nybg@archive.org
days_worked                        1193.0
first_day                      2016-11-29
last_day                       2020-03-06
Name: 3210, dtype: object
3210
-----------

In [346]:
operators['days_worked'].max()

6382.0

In [347]:
operators.loc[operators['days_worked'] == 6382.0]

,index,name,operator,days_worked,first_day,last_day
2095,8140976,Library of Congress,associate-mang-pau@archive.org,6382.0,2002-01-27,2019-07-19


In [348]:
operators.to_csv('/Volumes/Samsung_T5/scanning_labor_in_IA/days_worked_per_worker.csv')

## Average total days worked per worker across centers

In [357]:
df = operators[['name', 'days_worked']]

In [358]:
avg_days_worked = df.groupby(['name']).mean()

In [360]:
avg_days_worked.sort_values(by='days_worked')

,days_worked
name,
Zhejiang University,0.000000
Research Institute of Korean Studies,35.000000
Hamilton Public Library,44.000000
Servants of knowledge,82.500000
Hong Kong,100.135185
...,...
American Numismatic Society,1169.333333
New York Botanical Garden,1193.000000
Brown University,1478.857143


In [361]:
median_days_worked = df.groupby(['name']).median()

/Users/e.schwartz/anaconda3/envs/data_analyzer/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [364]:
median_days_worked.sort_values(by=['days_worked'], ascending=False)

,days_worked
name,
"BYU, Provo",1508.0
Brown University,1490.0
New York Botanical Garden,1193.0
American Printing House for the Blind,1114.5
The Ohio State University,1090.0
...,...
Hong Kong,10.0
Stanford University,0.0
Smithsonian Libraries and Archives,0.0
